# Lab - Intervalo de Confiança

* A partir o dataset IRIS (já trabalhado na aula), gere um novo dataset contendo apenas 1 das 4 medidas disponíveis no dataset original (Sepal Lentgh, Setal Width, Petal Length, Petal Width). Além disso, remova todas as linhas referentes a um dos tipos de flor e mantenha apenas 2 tipos ao invés dos 3 contidos no datset original (Setosa, Versicolour e Virginica). Ou seja, seu dataset conterá 1 coluna e 100 linhas (há 50 amostras de cada tipo de flor no datset original).

* Usando bootstrap para gerar o intervalo de confiança, calcule a diferença entre as médias para os dois tipos de flor.

* Informe o seu resultado em uma frase do tipo: "Com 95% de confiança, podemos afirmar que, em média, os cachorros são mais altos do que os gatos em 20,3cm, com intervalo de confiança de 17,2cm a 23,5cm".

* **Use o dataset inteiro** considerando que ele representa toda a amostra que você conseguiu coletar. Observe que isso é diferente do que fizemos em aula, onde consideramos que as 150 flores eram todas as existentes e nossa amostra continha uma quantidade menor (20, 30, por exemplo).



## Dados do(a) aluno(a)

**Nome**: Gabriel Paiva Medeiros

**Matrícula**: 118210822

## Imports.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.utils import resample

## Carregando o Datset Original

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
iris.data

In [4]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

## Gerando o novo Dataset

In [7]:
def transform_labels(label):
  if (label == 0):
    return "setosa"
  if (label == 1):
    return "versicolor"
  if (label == 2):
    return "virginica"

In [8]:
df = pd.DataFrame(data = iris.data)
df.columns = iris.feature_names
df['target'] = iris.target
df['target'] = df['target'].apply(lambda x: transform_labels(x))
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [9]:
data1 = df[df['target'] == 'setosa']['petal length (cm)']
data2 = df[df['target'] == 'versicolor']['petal length (cm)']
data = data1.to_numpy() - data2.to_numpy()
data = np.abs(data)

## Definição da função de Bootstrap

In [10]:
def bootstrap(data, func, alpha=0.95, n_rep=1000):
  
  theta_hat_error_list = list()
  theta = func(data)
  
  # Resampling
  for i in range(0,n_rep):
    x = resample(data, replace=True)
    theta_hat_error = func(x) - theta
    theta_hat_error_list.append(theta_hat_error)

  # Percentiles
  lower_p = ((1.0-alpha)/2.0) * 100
  upper_p = (alpha+((1.0-alpha)/2.0)) * 100

  # Lower and Upper Bounds
  lower_value = np.percentile(theta_hat_error_list, lower_p)
  upper_value = np.percentile(theta_hat_error_list, upper_p)

  lower_limit = theta+lower_value
  upper_limit = theta+upper_value

  return theta, lower_limit, upper_limit

## Análise das diferenças entre os dois tipos de flor escolhidos.

In [11]:
alpha = 0.95
theta = np.mean(data)
theta_bs, lower_limit, upper_limit = bootstrap(data, np.mean, alpha, 50)

print('Theta : %.3f' % theta_bs)
print('Confidence Interval (%.f%%): %.3f to %.3f' % (alpha*100, lower_limit, upper_limit))

contains_theta = (lower_limit <= theta_bs <= upper_limit)
print('Contains BS Theta:', contains_theta)

contains_theta = (lower_limit <= theta <= upper_limit)
print('Contains Real Theta:', contains_theta)

Theta : 2.798
Confidence Interval (95%): 2.644 to 2.936
Contains BS Theta: True
Contains Real Theta: True


Resultado:

Podemos afirmar que em média o comprimento das petalas de setosa são maiores que as pétalas da versicolor em 2.798 cm.
Isso com um intervalo de confiança de 95%.